This is my notebook file for playing around with new features, testing, etc.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sodapy import Socrata
import datetime
import numpy as np
import matplotlib.dates as mdates
import data_processing as dp
import definitions
import data_viz as dv
pd.set_option("display.max_rows", None)
import plotly.express as px
import definitions
import kaleido

In [4]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"
import dash
from dash import dcc
from dash import html

In [7]:
dp.get_data()

LOG: Fetched all raw data
LOG: Filtered Data
LOG: Added HHS Provisional data
LOG: Done getting data


In [9]:
dv.generate(path="../")
# #dp.test_data
# dp.get_us_positivity()
# # dp.get_state_positivity(["CA"])
# print(dp.test_data)

/Users/jonahdf/GitHub/covid-twitter-bot/src/data_processing.py:142: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.



FileNotFoundError: [Errno 2] No such file or directory: 'images/maps/rt.png'

In [142]:
state_hosps = dp.get_all_state_hosps()
hosp_fig = px.choropleth(state_hosps, locations='State', locationmode="USA-states", color="Hospitalizations per Million", scope="usa", color_continuous_scale='YlOrRd')
hosp_fig.update_layout(
    title={
        'text': "USA Hospitalizations per Million",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    coloraxis_colorbar_title="")
        
hosp_fig.show()
pio.kaleido.scope.default_scale = 10
hosp_fig.write_image("../images/maps/hosps.jpeg")
# case_fig = px.choropleth(state_cases, locations='State', locationmode="USA-states", color="Cases", scope="usa", color_continuous_scale='Purples')
# case_fig.show()

In [182]:
state_rt = dp.get_all_state_rt()
hosp_fig = px.choropleth(state_rt, locations='State', locationmode="USA-states", color="Rt", scope="usa", color_continuous_scale='YlOrRd')
hosp_fig.update_layout(
    title={
        'text': "USA Weekly Growth in Hospitalizations",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    coloraxis_colorbar_title="Rt")
pio.kaleido.scope.default_scale = 10
hosp_fig.write_image("../images/maps/rt.jpeg")

In [11]:
state_cases = pd.DataFrame(columns=['State', 'Cases'])
for state in definitions.states.keys():
    state_data = dp.get_state_cases([state], start_date=(pd.Timestamp.today() - pd.Timedelta(days=7)).date()) # adjust end date here
    if state_data.empty:
        cases = False
    else:
        cases = state_data[state_data['date'] == state_data['date'].max()].cases.values
        
        cases = cases[0]/definitions.populations[definitions.states[state]]*1000000
        state_cases = state_cases.append({"State":state, "Cases": float(cases)}, ignore_index=True)
case_fig = px.choropleth(state_cases, locations='State', locationmode="USA-states", color="Cases", scope="usa", color_continuous_scale='Purples')

In [14]:
app = dash.Dash(__name__)
app.layout = html.Div(children=[
    html.H1(children='Covid-Twitter-Bot-Dashboard'),

    html.Div(children='''
        Hospitalizations per capita
    ''', justify="center", align="center"),

    dcc.Graph(
    id='example-graph',
    figure=case_fig
    ),

    dcc.Graph(
    id='example-grap2h',
    figure=hosp_fig
    ),
    
])

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/



INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [06/Oct/2021 19:58:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2021 19:58:38] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1633284190.12.1.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2021 19:58:38] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1633284190.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2021 19:58:38] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1633284190.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2021 19:58:38] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1633284190.7.2.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2021 19:58:38] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1633284190.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Oct/2021 19:58:38] "GET /_dash-component-suites/dash/dcc/dash_core_com